In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
# Path to the uploaded file (this will be displayed by Kaggle after you upload the file)
file_path_train = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
file_path_test = '/kaggle/input/house-prices-advanced-regression-techniques/test.csv'

# Load the dataset
data = pd.read_csv(file_path_train)

# Display the first few rows of the dataset
data.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [3]:
missing_data = data.isnull().sum().sort_values(ascending=False)
missing_data = missing_data[missing_data > 0]
missing_data


PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
Electrical         1
dtype: int64

In [4]:
categorical_cols_with_na = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 
                            'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 
                            'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
                            'BsmtFinType2', 'MasVnrType', 'Electrical']

# Fill missing values with "None" for specified columns
#data[categorical_cols_with_na] = data[categorical_cols_with_na].fillna("None")

# Display the first few rows to ensure imputation is successful
data[categorical_cols_with_na].head()

,PoolQC,MiscFeature,Alley,Fence,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,MasVnrType,Electrical
0,NaN,NaN,NaN,NaN,NaN,Attchd,RFn,TA,TA,Gd,TA,No,GLQ,Unf,BrkFace,SBrkr
1,NaN,NaN,NaN,NaN,TA,Attchd,RFn,TA,TA,Gd,TA,Gd,ALQ,Unf,NaN,SBrkr
2,NaN,NaN,NaN,NaN,TA,Attchd,RFn,TA,TA,Gd,TA,Mn,GLQ,Unf,BrkFace,SBrkr
3,NaN,NaN,NaN,NaN,Gd,Detchd,Unf,TA,TA,TA,Gd,No,ALQ,Unf,NaN,SBrkr
4,NaN,NaN,NaN,NaN,TA,Attchd,RFn,TA,TA,Gd,TA,Av,GLQ,Unf,BrkFace,SBrkr


In [5]:
# Impute missing values for numerical features with the mean
data['LotFrontage'] = data['LotFrontage'].fillna(0)
data['MasVnrArea'] = data['MasVnrArea'].fillna(0)
data['GarageYrBlt'] = data['GarageYrBlt'].fillna(0)

# Display the first few rows to ensure imputation is successful
data[['LotFrontage', 'MasVnrArea', 'GarageYrBlt']].head()


,LotFrontage,MasVnrArea,GarageYrBlt
0,65.0,196.0,2003.0
1,80.0,0.0,1976.0
2,68.0,162.0,2001.0
3,60.0,0.0,1998.0
4,84.0,350.0,2000.0


In [6]:
# List of remaining categorical columns to fill with the mode
remaining_categorical_cols = ['Electrical']

# Fill missing values with the mode for specified columns
#data[remaining_categorical_cols] = data[remaining_categorical_cols].fillna(data.mode().iloc[0])

# Display the first few rows to ensure imputation is successful
data[remaining_categorical_cols].head()


,Electrical
0,SBrkr
1,SBrkr
2,SBrkr
3,SBrkr
4,SBrkr


In [7]:
# Check for remaining missing values we may have missed
missing_data = data.isnull().sum().sort_values(ascending=False)
missing_data = missing_data[missing_data > 0]
missing_data

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
GarageType        81
GarageQual        81
GarageCond        81
GarageFinish      81
BsmtExposure      38
BsmtFinType2      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
Electrical         1
dtype: int64

In [8]:
# Encode categorical variables
data = pd.get_dummies(data)

# Display the first few rows to ensure encoding is successful
data.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,False,False,False,True,False,False,False,False,True,False
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,False,False,False,True,False,False,False,False,True,False
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,False,False,False,True,False,False,False,False,True,False
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,False,False,False,True,True,False,False,False,False,False
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,False,False,False,True,False,False,False,False,True,False


In [9]:
# Feature engineering: creating Age feature
data['Age'] = data['YrSold'] - data['YearBuilt']

# Display the first few rows to ensure the new feature is added
data[['YrSold', 'YearBuilt', 'Age']].head()

,YrSold,YearBuilt,Age
0,2008,2003,5
1,2007,1976,31
2,2008,2001,7
3,2006,1915,91
4,2008,2000,8


In [10]:
# Dealing with outliers

from scipy.stats import zscore

# Calculate Z-scores for numerical features
numerical_features = data.select_dtypes(include=['int64', 'float64']).columns
z_scores = data[numerical_features].apply(zscore)

# Identify outliers (Z-score > 3 or Z-score < -3)
outliers = (z_scores > 3) | (z_scores < -3)

# Get a boolean array indicating rows with any outliers
outlier_indices = outliers.any(axis=1)

outliers.head(20)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,Age
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
# Remove outliers
data_cleaned = data[~outlier_indices].copy()
#data_cleaned = data

In [12]:
# Fit the StandardScaler to the target variable 'SalePrice'
scaler_target = StandardScaler()
y_scaled = scaler_target.fit_transform(data_cleaned[['SalePrice']])

# Replace the original 'SalePrice' with the scaled values in the DataFrame
data_cleaned['SalePrice'] = y_scaled

# Normalize numerical features
numerical_features = data_cleaned.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
data_cleaned[numerical_features] = scaler.fit_transform(data_cleaned[numerical_features])

# Display the first few rows to ensure normalization is successful
data_cleaned.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Age
0,-1.737427,0.153859,0.279179,-0.222441,0.665812,-0.533683,0.938100,0.799130,0.841391,0.699284,...,False,False,True,False,False,False,False,True,False,-0.929922
2,-1.732700,0.153859,0.373162,0.533591,0.665812,-0.533683,0.867618,0.750252,0.583050,0.157771,...,False,False,True,False,False,False,False,True,False,-0.859596
4,-1.727972,0.153859,0.874404,1.346326,1.441663,-0.533683,0.832378,0.652495,2.011520,0.573752,...,False,False,True,False,False,False,False,True,False,-0.824433
6,-1.723244,-0.850031,0.592455,0.218758,1.441663,-0.533683,0.973341,0.896886,0.765408,2.331208,...,False,False,True,False,False,False,False,True,False,-1.000249
10,-1.713789,-0.850031,0.435817,0.520091,-0.885889,-0.533683,-0.401049,-1.058239,-0.647866,1.191569,...,False,False,True,False,False,False,False,True,False,0.406272


In [13]:
from sklearn.model_selection import train_test_split, GridSearchCV

# Separate target variable and features
X = data_cleaned.drop(['SalePrice'], axis=1)
y = data_cleaned['SalePrice']

# Splitting the data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=300, random_state=42)
#rf_model = LinearRegression()

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the validation set
y_pred_scaled = rf_model.predict(X_val)

# Inverse transform the scaled predictions and actual values
y_pred = scaler_target.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
y_val_original = scaler_target.inverse_transform(y_val.values.reshape(-1, 1)).flatten()

# Looking at prediction vs actual
for i in range (90, 100):
    print("prediction:", y_pred[i], "actual:", y_val_original[i], "loss:", abs(y_pred[i]-y_val_original[i]))

# Calculate the Mean Squared Error
mse = mean_squared_error(y_val_original, y_pred)
print(f"Mean Squared Error: {mse}")

# Optional: Calculate the Root Mean Squared Error
rmse = mse ** 0.5
print(f"Root Mean Squared Error: {rmse}")

# Calculate the R² Score on the original scale
r2 = r2_score(y_val_original, y_pred)
print(f"R² Score: {r2}")

prediction: 122350.09333333338 actual: 101999.99999999999 loss: 20350.093333333396
prediction: 114836.35999999996 actual: 109999.99999999999 loss: 4836.3599999999715
prediction: 234088.13333333336 actual: 254900.0 loss: 20811.86666666664
prediction: 242650.0666666667 actual: 287090.0 loss: 44439.93333333329
prediction: 209992.53333333333 actual: 215000.0 loss: 5007.466666666674
prediction: 151005.73999999996 actual: 145250.0 loss: 5755.739999999962
prediction: 188247.63666666666 actual: 225000.0 loss: 36752.36333333334
prediction: 130865.78000000006 actual: 132500.0 loss: 1634.219999999943
prediction: 166338.40333333335 actual: 174000.0 loss: 7661.59666666665
prediction: 176319.8933333333 actual: 187500.0 loss: 11180.106666666688
Mean Squared Error: 397123162.4666644
Root Mean Squared Error: 19927.94927900672
R² Score: 0.8957720222889415


In [15]:
# Loading the test dataset
dataTest = pd.read_csv(file_path_test)
dataTest.head()

# Save Id from being normalized for future purposes
realID = dataTest['Id']

In [16]:
# Cleaning the test data set
missing_dataTest = dataTest.isnull().sum().sort_values(ascending=False)
missing_dataTest = missing_dataTest[missing_dataTest > 0]

# Get the data types of columns with missing values
missing_dataTest_types = dataTest[missing_dataTest.index].dtypes

# Combine the missing data counts and data types into a single DataFrame
missing_dataTest_df = pd.DataFrame({
    'Missing Values': missing_dataTest,
    'Data Type': missing_dataTest_types
})

print(missing_dataTest_df)

              Missing Values Data Type
PoolQC                  1456    object
MiscFeature             1408    object
Alley                   1352    object
Fence                   1169    object
MasVnrType               894    object
FireplaceQu              730    object
LotFrontage              227   float64
GarageYrBlt               78   float64
GarageQual                78    object
GarageFinish              78    object
GarageCond                78    object
GarageType                76    object
BsmtCond                  45    object
BsmtQual                  44    object
BsmtExposure              44    object
BsmtFinType1              42    object
BsmtFinType2              42    object
MasVnrArea                15   float64
MSZoning                   4    object
BsmtHalfBath               2   float64
Utilities                  2    object
Functional                 2    object
BsmtFullBath               2   float64
BsmtFinSF1                 1   float64
BsmtFinSF2               

In [17]:
categorical_cols_with_na = missing_dataTest_df[missing_dataTest_df['Data Type'] == 'object'].index

# Fill missing values with "None" for specified columns
#dataTest[categorical_cols_with_na] = dataTest[categorical_cols_with_na].fillna("None")

# Display the first few rows to ensure imputation is successful
dataTest[categorical_cols_with_na].head()

,PoolQC,MiscFeature,Alley,Fence,MasVnrType,FireplaceQu,GarageQual,GarageFinish,GarageCond,GarageType,...,BsmtExposure,BsmtFinType1,BsmtFinType2,MSZoning,Utilities,Functional,KitchenQual,Exterior2nd,Exterior1st,SaleType
0,NaN,NaN,NaN,MnPrv,NaN,NaN,TA,Unf,TA,Attchd,...,No,Rec,LwQ,RH,AllPub,Typ,TA,VinylSd,VinylSd,WD
1,NaN,Gar2,NaN,NaN,BrkFace,NaN,TA,Unf,TA,Attchd,...,No,ALQ,Unf,RL,AllPub,Typ,Gd,Wd Sdng,Wd Sdng,WD
2,NaN,NaN,NaN,MnPrv,NaN,TA,TA,Fin,TA,Attchd,...,No,GLQ,Unf,RL,AllPub,Typ,TA,VinylSd,VinylSd,WD
3,NaN,NaN,NaN,NaN,BrkFace,Gd,TA,Fin,TA,Attchd,...,No,GLQ,Unf,RL,AllPub,Typ,Gd,VinylSd,VinylSd,WD
4,NaN,NaN,NaN,NaN,NaN,NaN,TA,RFn,TA,Attchd,...,No,ALQ,Unf,RL,AllPub,Typ,Gd,HdBoard,HdBoard,WD


In [18]:
numerical_cols_with_na = missing_dataTest_df[missing_dataTest_df['Data Type'] == 'float64'].index
print(numerical_cols_with_na)

# Impute missing values for numerical features with the median
dataTest['LotFrontage'] = dataTest['LotFrontage'].fillna(0)
dataTest['GarageYrBlt'] = dataTest['GarageYrBlt'].fillna(0)
dataTest['MasVnrArea'] = dataTest['MasVnrArea'].fillna(0)
dataTest['BsmtHalfBath'] = dataTest['BsmtHalfBath'].fillna(0)
dataTest['BsmtFullBath'] = dataTest['BsmtFullBath'].fillna(0)
dataTest['BsmtFinSF1'] = dataTest['BsmtFinSF1'].fillna(0)
dataTest['BsmtFinSF2'] = dataTest['BsmtFinSF2'].fillna(0)
dataTest['BsmtUnfSF'] = dataTest['BsmtUnfSF'].fillna(0)
dataTest['TotalBsmtSF'] = dataTest['TotalBsmtSF'].fillna(0)
dataTest['GarageCars'] = dataTest['GarageCars'].fillna(0)
dataTest['GarageArea'] = dataTest['GarageArea'].fillna(0)

# Display the first few rows to ensure imputation is successful
dataTest[['LotFrontage', 'GarageYrBlt', 'MasVnrArea', 'BsmtHalfBath',
       'BsmtFullBath', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'GarageCars', 'GarageArea']].head()

Index(['LotFrontage', 'GarageYrBlt', 'MasVnrArea', 'BsmtHalfBath',
       'BsmtFullBath', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'GarageCars', 'GarageArea'],
      dtype='object')


,LotFrontage,GarageYrBlt,MasVnrArea,BsmtHalfBath,BsmtFullBath,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,GarageCars,GarageArea
0,80.0,1961.0,0.0,0.0,0.0,468.0,144.0,270.0,882.0,1.0,730.0
1,81.0,1958.0,108.0,0.0,0.0,923.0,0.0,406.0,1329.0,1.0,312.0
2,74.0,1997.0,0.0,0.0,0.0,791.0,0.0,137.0,928.0,2.0,482.0
3,78.0,1998.0,20.0,0.0,0.0,602.0,0.0,324.0,926.0,2.0,470.0
4,43.0,1992.0,0.0,0.0,0.0,263.0,0.0,1017.0,1280.0,2.0,506.0


In [19]:
# Check for remaining missing values we may have missed
missing_dataTest = dataTest.isnull().sum().sort_values(ascending=False)
missing_dataTest = missing_dataTest[missing_dataTest > 0]
missing_dataTest

PoolQC          1456
MiscFeature     1408
Alley           1352
Fence           1169
MasVnrType       894
FireplaceQu      730
GarageFinish      78
GarageCond        78
GarageQual        78
GarageType        76
BsmtCond          45
BsmtExposure      44
BsmtQual          44
BsmtFinType2      42
BsmtFinType1      42
MSZoning           4
Utilities          2
Functional         2
SaleType           1
Exterior2nd        1
KitchenQual        1
Exterior1st        1
dtype: int64

In [20]:
# Encode categorical variables
dataTest = pd.get_dummies(dataTest)

# Ensure that the test dataset has the same columns as the training dataset
dataTest = dataTest.reindex(columns=X_train.columns, fill_value=0)

# Display the first few rows to ensure encoding is successful
dataTest.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Age
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,False,False,True,False,False,False,False,True,False,0
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,False,False,True,False,False,False,False,True,False,0
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,False,False,True,False,False,False,False,True,False,0
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,False,False,True,False,False,False,False,True,False,0
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,False,False,True,False,False,False,False,True,False,0


In [21]:
# Feature engineering: creating Age feature
dataTest['Age'] = dataTest['YrSold'] - dataTest['YearBuilt']

# Display the first few rows to ensure the new feature is added
dataTest[['YrSold', 'YearBuilt', 'Age']].head()

,YrSold,YearBuilt,Age
0,2010,1961,49
1,2010,1958,52
2,2010,1997,13
3,2010,1998,12
4,2010,1992,18


In [22]:
# Dealing with outliers

from scipy.stats import zscore

# Calculate Z-scores for numerical features
numerical_features = dataTest.select_dtypes(include=['int64', 'float64']).columns
z_scores = dataTest[numerical_features].apply(zscore)

# Identify outliers (Z-score > 3 or Z-score < -3)
outliers = (z_scores > 3) | (z_scores < -3)

# Get a boolean array indicating rows with any outliers
outlier_indices = outliers.any(axis=1)

outliers.head(20)

# Remove outliers
#dataTest_cleaned = dataTest[~outlier_indices].copy()

# Can't delete outliers in a test set so...
dataTest_cleaned = dataTest

In [23]:
# Normalize numerical features
numerical_features = dataTest.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
dataTest_cleaned[numerical_features] = scaler.fit_transform(dataTest_cleaned[numerical_features])

# Display the first few rows to ensure normalization is successful
dataTest_cleaned.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Age
0,-1.730864,-0.874711,0.684849,0.363929,-0.751101,0.400766,-0.340945,-1.072885,-0.563316,0.063936,...,False,False,True,False,False,False,False,True,False,0.413784
1,-1.728490,-0.874711,0.715852,0.897861,-0.054877,0.400766,-0.439695,-1.214908,0.047057,1.063714,...,False,False,True,False,False,False,False,True,False,0.512398
2,-1.726115,0.061351,0.498831,0.809646,-0.751101,-0.497418,0.844059,0.678742,-0.563316,0.773668,...,False,False,True,False,False,False,False,True,False,-0.769577
3,-1.723741,0.061351,0.622843,0.032064,-0.054877,0.400766,0.876976,0.678742,-0.450284,0.358376,...,False,False,True,False,False,False,False,True,False,-0.802448
4,-1.721367,1.465443,-0.462261,-0.971808,1.337571,-0.497418,0.679475,0.394694,-0.563316,-0.386513,...,False,False,True,False,False,False,False,True,False,-0.605221


In [24]:
# I want data to be called X_test so...
X_test = dataTest_cleaned

# Creating a final model
rf_model_final = RandomForestRegressor(n_estimators=300)

# Train the model
rf_model_final.fit(X, y)

# Predict on the validation set
y_test_scaled = rf_model_final.predict(X_test)

# Inverse transform the scaled predictions and actual values
y_test = scaler_target.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()

# Create the submission DataFrame
submission = pd.DataFrame({
    'Id': realID,
    'SalePrice': y_test
})
print(submission)

        Id      SalePrice
0     1461  124340.636667
1     1462  154743.966667
2     1463  177908.440000
3     1464  176332.570000
4     1465  194503.556667
...    ...            ...
1454  2915   96342.360000
1455  2916   98561.843333
1456  2917  151719.150000
1457  2918  127050.036667
1458  2919  237531.220000

[1459 rows x 2 columns]


In [25]:
# Exporting submission to a csv
submission.to_csv('/kaggle/working/submission.csv', index=False)